In [1]:
import xarray as xr   # xarray pacckage for netcdf io and data processing
import glob           # allow unix shell like text
import pandas as pd
import numpy as np
import datetime as dt

import netCDF4
import locale          # This is needed to read number in different language or region format correctly

locale.setlocale(locale.LC_ALL, 'nn_NO.utf8') 

xr.set_options(display_style="html")
%matplotlib inline

In [ ]:
sites=["ALP1","ALP2","ALP3","ALP4","SUB1","SUB2","SUB3","SUB4","BOR1","BOR2","BOR3","BOR4"]
siteID1=["Ulvehaugen","Lavisdalen","Gudmedalen","Skjelingahaugen","Alrust","Hogsete","Rambera","Veskre","Fauske","Vikesland","Arhelleren","Ovstedalen"] 
siteID2=["Ulvhaugen","Lavisdalen","Gudmedalen","Skjellingahaugen","Alrust","Hogsete","Rambera","Veskre","Fauske","Vikesland","Arhelleren","Ovstedalen"]
siteID3=["ULV","LAV","GUD","SKJ","ALR","HOG","RAM","VES","FAU","VIK","ARH","OVS"]

blockID=[['Ulv1','Ulv2','Ulv3','Ulv4'],
         ['Lav1','Lav2','Lav3','Lav4'],
         ['Gud12','Gud13','Gud15','Gud5'], #12 13 15 5
         ['Skj1','Skj2','Skj3','Skj4'],
         ['Alr1','Alr2','Alr3','Alr5'], #1, 2,3,5
         ['Hog1','Hog2','Hog3','Hog4'],
         ['Ram4','Ram5','Ram6','Ram8'], #4 5 6 8
         ['Ves1','Ves2','Ves3','Ves4'],
         ['Fau1','Fau2','Fau4','Fau5'], #1 2 4 5
         ['Vik2','Vik3','Vik4','Vik5'], #2, 3,4,5
         ['Arh1','Arh2','Arh3','Arh4'],       
         ['Ovs1','Ovs2','Ovs3','Ovs4']] 

In [ ]:
# Read observation data

plant_comp=pd.read_csv("/home/huitang/Documents/MossLichen_testbed/data_raw/FunCaB_clean_composition_2015-2019.csv", index_col=None, header=0)

# File contain soil organic matter info
soil_org=pd.read_csv("/home/huitang/Documents/MossLichen_testbed/data_raw/soilCN_2015.txt", index_col=None, header=0, sep='\t')

soil_bd=pd.read_csv("/home/huitang/Documents/MossLichen_testbed/data_raw/soilBD.txt", index_col=None, header=0, sep='\t').dropna()

# File contain soil depth and texture, etc
soil_info=pd.read_csv("/home/huitang/Documents/MossLichen_testbed/data_raw/Soil_structure_2013-2018_clean.csv", index_col=None, header=0)

# Slop data (so far missing)

In [ ]:
# Modify surface data file

for i in range(11,12):

    soil_org_sel=soil_org[(soil_org["siteID"]==siteID3[i])
                             & (soil_org["blockID"].isin(['1','2','3','4']))]
    soil_bd_sel=soil_bd[(soil_bd["siteID"]==siteID3[i])] 
    soil_info_sel=soil_info[(soil_info["siteID"]==siteID1[i]) & (soil_info["variable"]=="soil_depth")]
    
    # Organic matter#
    bd_obs=soil_bd_sel[soil_bd_sel["depth"]=="8"]["BD"].str.replace(',','.').astype(float).mean()  #["BD"].mean()
    c_obs=soil_org_sel[soil_org_sel["depth"]==10]["C"].str.replace(',','.').astype(float).mean()
    org_obs=bd_obs*c_obs/100/1000*1000000/0.58     # Bulk density: g/cm3, Carbon content: %, organic: kg/m3
    org_obs=min(110,org_obs)      # model assume less than 130 kg/m3 organic matter 
    print(org_obs)

    # Soil depth#
    sd_obs=soil_info_sel["value"].mean()  # cm
    print(sd_obs)
    
    # Slope (missing here)#
    # Soil texture (missing here)
    
    # Plant cover #
    # read corresponding sites
    plant_comp_sel=plant_comp[(plant_comp["siteID"]==siteID1[i]) & (plant_comp["treatment"]=="GF")
                             & (plant_comp["blockID"].isin(blockID[i])) & (plant_comp["year"]==2015)]
    # Calculating plant cover for sepcific experiments
    # You can change this according to your purpose
    plant_cover_obs=(plant_comp_sel[plant_comp["blockID"]==blockID[i][0]].iloc[0]["total_bryophytes"]
                 +plant_comp_sel[plant_comp["blockID"]==blockID[i][1]].iloc[0]["total_bryophytes"]
                 +plant_comp_sel[plant_comp["blockID"]==blockID[i][2]].iloc[0]["total_bryophytes"]
                 +plant_comp_sel[plant_comp["blockID"]==blockID[i][3]].iloc[0]["total_bryophytes"])/4

    plant_cover_obs=min(plant_cover_obs,100.0)    # make sure the total coverage is below 100

    # Plant height #
    plant_comp_sel=plant_comp[(plant_comp["siteID"]==siteID1[i]) & (plant_comp["treatment"]=="GF")
                             & (plant_comp["blockID"].isin(blockID[i])) & (plant_comp["year"]==2017)]
    # Calculating plant cover for sepcific experiments
    # You can change this according to your purpose
    plant_height_obs=(
                  plant_comp_sel[plant_comp["blockID"]==blockID[i][0]].iloc[0]["moss_height"]
                 +plant_comp_sel[plant_comp["blockID"]==blockID[i][1]].iloc[0]["moss_height"]
                 +plant_comp_sel[plant_comp["blockID"]==blockID[i][2]].iloc[0]["moss_height"]
                 +plant_comp_sel[plant_comp["blockID"]==blockID[i][3]].iloc[0]["moss_height"]
                  )/4
    
    # Modify surface data file #
    # Copy a defaut surface data file for modification
    # Can downloaded from here: https://ns2806k.webs.sigma2.no/EMERALD/EMERALD_platform/inputdata_fates_platform/
    
    # open surface data file to modify
    dset = netCDF4.Dataset('/home/huitang/saga/work/inputdata/lnd/clm2/surfdata_map/'+sites[i]+'/surfdata_'+sites[i]+'_simyr2000.nc', 'r+')
    # modify land cover
    dset['PCT_NAT_PFT'][0,:,:] = 100-plant_cover_obs # pay attention to the index, 0: barren ground
    dset['PCT_NAT_PFT'][1:12,:,:] = 0          
    dset['PCT_NAT_PFT'][12,:,:] = plant_cover_obs      # 12: grass
    dset['PCT_NAT_PFT'][13:15,:,:] = 0
    dset['PCT_NATVEG'][:,:] = 100
    dset['PCT_CROP'][:,:] = 0
    dset['PCT_CFT'][:,:,:] = 0
    dset['PCT_WETLAND'][:,:] = 0
    dset['PCT_LAKE'][:,:] = 0
    dset['PCT_GLACIER'][:,:] = 0
    dset['PCT_URBAN'][:,:,:] = 0
    # Modify soil properties
    dset['ORGANIC'][0:3,:,:] = org_obs        # the layers of soil to modify depending on the availability of the data
    #dset['PCT_SAND'][:,:,:] = 0
    #dset['PCT_CLAY'][:,:,:] = 0
    dset['zbedrock'][:,:] = sd_obs/100       # Modify soil depth:
    dset['SLOPE'][:,:] = 20.0
    
    # Modify satellite phenology (only if you use reduced complexity mode)
    #dset['MONTHLY_LAI'][:,:,:,:] = 0
    #dset['MONTHLY_SAI'][:,:,:,:] = 0
    dset['MONTHLY_HEIGHT_TOP'][:,12,:,:] = plant_height_obs/1000
    dset['MONTHLY_HEIGHT_BOT'][:,12,:,:] = 0.001/1000
    # Modify topography

    dset.close()